In [4]:
import sys 
sys.path = ['/home/roboy/anaconda3/envs/roboy/lib/python3.6/site-packages'] + sys.path
sys.path.append('/home/roboy/vision_workspace/Vision/')

In [5]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os, pickle
import re
import signal
import operator
from sklearn import svm
from sklearn import tree

In [16]:
from matplotlib import pyplot as plt

## Rearrange Embeddings for training

Rearrange data to be compatible for training with sklearn

In [6]:
def rearrange_people_reps(people_reps):
    all_reps = []
    classes = []
    for person in range(len(people_reps)):
        for rep in range(len(people_reps[person])):
            all_reps.append(people_reps[person][rep])
            classes.append(person)
    return all_reps, classes            

## Load embeddings

Load all Representation of own dataset (To be Created with Facenet_Embeddings.ipyn)

In [9]:
names = np.load('../models/lfw_embeddings/facenet_names.npy')
people_reps = np.load('../models/lfw_embeddings/facenet_reps.npy')

Reshape for training

In [10]:
reps, classes = rearrange_people_reps(people_reps)

## Train SVM and save to file

In [19]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [20]:
cls = OneVsRestClassifier(LinearSVC(random_state=0))

In [21]:
cls.fit(reps, classes)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [24]:
save_dir = '../models/SVM/'
os.makedirs(os.path.dirname(save_dir), exist_ok=True)
pickle.dump( cls, open( save_dir + 'svm_lfw.mod' , "wb" ) )

## Train Decision Tree and save to file

In [8]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(reps, classes)

In [9]:
save_dir = '../models/Tree/'
os.makedirs(os.path.dirname(save_dir), exist_ok=True)
pickle.dump( clf, open( save_dir + 'own.b' , "wb" ) )

--> TODO: Try Tensorflow Model (e.g. 2 fully connected layers) to map from embeddings to classes